### Setup

In [12]:
!pip install -U -q PyDrive
!pip install -U plotly-express
!pip install -U plotly

Requirement already up-to-date: plotly-express in /usr/local/lib/python3.6/dist-packages (0.4.1)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.11.0)


importação das bibiotecas  
Pandas: Explorar, tratar e transformar os dados  
Numpy: Usada como base no Pandas, otimizada para operações numericas  
Pydrive: Abstração da API do Google Drive  
Plotly: geração rápidade de gráficos interativos

In [13]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import requests
import plotly.express as px
from matplotlib import pyplot as plt

Autorizar o acesso à API do Google drive (os arquivos são públicos)

In [14]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Explorar dados

Função que vai tratar cada dataframe (parte de 1mi de linhas do arquivo). Primeiro as colunas são escolhidas e renomeadas e depois os tipos de dados são modificados

In [15]:
def chunk_preprocessing(chunk):
    # ---------------------------------------
    # ------------ Map columns --------------
    # ---------------------------------------
    # Definição das colunas que serão usadas, com seus respectivos renomeamentos
    # O que não tiver listado não será usado
    mapper = {
        'MÊS DISPONIBILIZAÇÃO': 'mes', 
        'UF': 'uf', 
        'ENQUADRAMENTO': 'enquadramento',
        'NOME MUNICÍPIO': 'municipio',
        'PARCELA': 'parcela', 
        'OBSERVAÇÃO': 'observacao', 
        'VALOR BENEFÍCIO': 'valor'
    }
    # Renomeia as colunas e remove as não listadas no dicionario acima
    for column in chunk.columns:
        try:
            chunk.rename({column: mapper[column]}, inplace=True, axis=1) 
        except KeyError:
            chunk.drop(column, axis=1, inplace=True)
    # ---------------------------------------
    # ------------- Discart data ------------
    # ---------------------------------------
    chunk.dropna(inplace=True)
    # ---------------------------------------
    # ------------ Convert types ------------
    # ---------------------------------------
    # Muda os tipos de dados para otimizar o uso de memória e fazer operações
    # sobre os mesmos
    chunk.parcela = chunk.parcela.str.extract(r'(\d+)', expand=False).astype(np.uint8)
    chunk.mes = chunk.mes.astype(str).str.extract(r"\d{4}(\d{2})", expand=False).astype(np.uint8)
    chunk.valor = chunk.valor.str.extract(r"(\d+)", expand=False).astype(np.uint16)
    chunk.uf = chunk.uf.astype('category')
    chunk.enquadramento = chunk.enquadramento.astype('category')
    chunk.observacao = chunk.observacao.astype('category')

Essa parte lista os arquivos dentro da pasta csv

In [16]:
# load file details from folder
id = '16AyG_SGDXynaCEfFYJrsTJbsBWbeDqik'
file_list = drive.ListFile({'q': f"'{id}' in parents and trashed=false"}).GetList()

#### Fazer dataframe indexado por estado

Essa parte define a função que gera o dataframe com as estatistíscas para cada estado

In [17]:
states = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados").json()
def create_state_df():
    states_dict = {state['sigla']:state for state in states}
    siglas = list(states_dict.keys())
    nomes = [states_dict[sigla]['nome'] for sigla in siglas]
    codigos = [states_dict[sigla]['id'] for sigla in siglas]
    result = pd.DataFrame({
        'sum_pessoas': np.zeros((27,), np.int32), 
        'sum_valor': np.zeros((27,), np.int64),
        'sum_devolucoes': np.zeros((27,), np.int32),
        'sum_devolvido': np.zeros((27,), np.int64),
        'cod_ibge': codigos,
        'population': np.zeros((27,), np.int32),
        'nome': nomes
        }, index=siglas)
    result.cod_ibge = result.cod_ibge.astype(np.int8)
    # Preeecher dados já conhecidos
    for state in states: 
        link = f"https://servicodados.ibge.gov.br/api/v3/agregados/1292/variaveis/610?localidades=N3[{state['id']}]"
        response = requests.get(link).json()
        result.iloc[result.index.get_loc(state['sigla']), result.columns.get_loc('population')] = np.int32(
            response[0]['resultados'][0]['series'][0]['serie']['2010'])
    return result

Essa parte processa as partes de cada arquivo que será usado

In [18]:
df_state_original = create_state_df()
df_state = df_state_original.copy()
i = 0
for f in file_list:
    # Referencia para o arquivo
    downloaded = drive.CreateFile({'id':f['id']})
    # Baixa o conteudo do arquivo
    downloaded.GetContentFile(f['title'])  
    # Inicia um iterador que será usado para percorrer as chunks (pedaços) do
    # arquivo, estamos usando um tamanho de 1000000 de linhas para cada chunk.
    # Cada chunk é um dataframe.
    df_chunk = pd.read_csv(f['title'], encoding='latin1', sep=';', chunksize=1000000)
    for chunk in df_chunk:
        print(f"processing chunk {i}")
        i += 1
        # Chama a função criada criada para tratar os dados
        chunk_preprocessing(chunk)
        # Cria o dataframe com as estatisticas
        temp_df = df_state_original.copy()
        # Agrupa os dados por estado
        group = chunk.groupby(by='uf')
        # O total de pessoas é a quantidade de linhas de cada grupo
        temp_sum_pessoas = group.size()
        # Somatorio do valor de auxilio de todas as pessoas que receberam
        temp_sum_valor = group.valor.sum()
        # Incrementa as novas informações no dataframe com as estatisticas
        if(f == file_list[-1]):
            temp_df['sum_pessoas'] = df_state['sum_pessoas'].add(
                temp_sum_pessoas, fill_value=0).astype(np.int32)
        temp_df['sum_valor'] = df_state['sum_valor'].add(
            temp_sum_valor, fill_value=0).astype(np.int64)
        # Mesma coisa, só que para as devoluções, essa parte está sendo revisada
        temp_sum_devolucoes = None
        if (group.ngroups == 1):
            temp_sum_devolucoes = group.apply(lambda g: (
                g.observacao == "Valor devolvido à União")).sum(axis=1).astype(np.int64)
        else:
            temp_sum_devolucoes = group.apply(lambda g: (
                g.observacao == "Valor devolvido à União")).groupby('uf').sum().astype(np.int64)
        temp_df['sum_devolucoes'] = df_state['sum_devolucoes'].add(
            temp_sum_devolucoes, fill_value=0).astype(np.int32)
        temp_sum_devolvido = None
        if (group.ngroups == 1):
            temp_sum_devolvido = group.apply(lambda g: (
                g.observacao == "Valor devolvido à União") * 
                g.valor * g.parcela).sum(axis=1).astype(np.int64)
        else:
            temp_sum_devolvido = group.apply(lambda g: (
                g.observacao == "Valor devolvido à União") * 
                g.valor * (g.parcela-1)).groupby('uf').sum().astype(np.int64)
        temp_df['sum_devolvido'] = df_state['sum_devolvido'].add(
            temp_sum_devolvido, fill_value=0).astype(np.int64)
        df_state = temp_df.combine_first(df_state)

processing chunk 0
processing chunk 1
processing chunk 2
processing chunk 3


KeyboardInterrupt: ignored

Calculando a quantidade per capita de cada estado

In [ ]:
df_state['per_population'] = (df_state.sum_pessoas / df_state.population) * 100
df_state['nome_and_per_population'] = df_state.nome + df_state.per_population.map(lambda x: f" ({x:.2f}%)")
# Ordenando da maior pra menor quantidade per capita por estado e colocando em outro DF
df_biggest = df_state.sort_values('per_population', ascending=False)
df_biggest

#### Gráfico de barras com todos os estados em ordem decrescente por auxílio per capita. 


In [ ]:
plt.figure(figsize=(40,20))
plt.bar(df_biggest.nome, df_biggest.per_population, color='blue', align='center')
plt.xticks(rotation=90)
plt.xlabel('Estados')
plt.ylabel('Quantidade per capita por estado (em %)')
plt.title('Qauntidade per capita por estado')
plt.show()

In [ ]:
# Selecionando os 5 estados com maior quantidade de auxilio per capita
df_biggest = df_state.sort_values('per_population', ascending=False).head()

# Criando o grafico
plt.figure(figsize=(20,10))
plt.bar(df_biggest.nome, df_biggest.per_population, color='blue', align='center')
plt.xlabel('Estados')
plt.ylabel('Quantidade per capita por estado (em %)')
plt.title('5 estados com maior quantidade de auxilio per capita')
plt.show()

#### Fazer mapa com pessoas que receberam em cada estado 

Esse código utilizada o gráfico choropleth do ploty express para gerar um mapa interativo com todos os estados do Brasil. Os outros mapas podem ser gerados trocando os parametros, e podem ser visualizados em http://datapane.com/u/pablo

In [ ]:
# Esse é um arquivo com dos dados de distribução geografica do brasil fornecido pelo IBGE
brazil = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/?formato=application/vnd.geo+json&resolucao=2").json()
# Comando para criar a fugura
fig = px.choropleth(df_state, geojson=brazil, color='per_population',
                    locations='cod_ibge', color_continuous_scale='greens',
                    hover_name='nome_and_per_population', featureidkey='properties.codarea',
                    range_color=(0, max(df_state.per_population)),
                    center={"lat": -5.78362, "lon": -36.914},
                    hover_data={
                        'sum_valor': True, 'cod_ibge': False, 'sum_devolvido': True, 
                        'sum_pessoas': True, 'sum_devolucoes': True, 'population':True,
                        'per_population': True
                        },
                    labels = {
                        'sum_valor': 'Valor total recebido',
                        'sum_devolvido': 'Valor total devolvido',
                        'sum_devolucoes': 'Total de pessoas que devolveram',
                        'sum_pessoas': 'Total de pessoas que receberam',
                        'population': 'População total do estado',
                        'per_population': 'Porcentagem da população que recebeu'
                        },
                    title = "Porcentagem da população (%) beneficiada pelo auxílio emergencial no Brasil em cada estado"
                    )
# Configuração de geometria da figura 
fig.update_geos(projection={"scale": 30}, 
                fitbounds="locations", 
                visible=False,
                )
# Configuração de layout da figura 
fig.update_layout(coloraxis_colorbar_title = "Beneficiados (%)",
                  coloraxis_colorbar_showticklabels = True)
# Mostrar a figura
fig.show()